In [1]:
import pandas as pd
import numpy as np
import data_io
from data_io import space_time_series as st
import forecast_time_series
from forecast_time_series import forecast
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from statsmodels.tsa.seasonal import seasonal_decompose
from stldecompose import decompose, forecast
from stldecompose.forecast_funcs import (naive, drift, mean, seasonal_naive)

/home/mike/anaconda3/lib/python3.8/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pos = pd.read_excel('../data/PoS_serieT.xlsx',
                    encoding='utf-8', sheet_name='Sheet1', error_bad_lines=False)

In [3]:
pos

,Date,SentimentPolarity
0,2019-03-18,2.367347
1,2019-03-22,2.200000
2,2019-03-23,2.333333
3,2019-03-24,2.000000
4,2019-03-25,2.250000
...,...,...
370,2020-04-14,3.000000
371,2020-04-15,2.000000
372,2020-04-16,2.000000
373,2020-04-17,3.000000


In [4]:
pos['year'] = pd.to_datetime(pos.Date).dt.year
pos['month'] = pd.to_datetime(pos.Date).dt.month
pos['day'] = pd.to_datetime(pos.Date).dt.day

In [5]:
pos

,Date,SentimentPolarity,year,month,day
0,2019-03-18,2.367347,2019,3,18
1,2019-03-22,2.200000,2019,3,22
2,2019-03-23,2.333333,2019,3,23
3,2019-03-24,2.000000,2019,3,24
4,2019-03-25,2.250000,2019,3,25
...,...,...,...,...,...
370,2020-04-14,3.000000,2020,4,14
371,2020-04-15,2.000000,2020,4,15
372,2020-04-16,2.000000,2020,4,16
373,2020-04-17,3.000000,2020,4,17


In [6]:
index_time = pd.date_range(pos.Date[0], pos.Date[len(pos) -1], freq='D')

In [7]:
df_sp= pd.DataFrame(data={'SentimentPolarity':np.zeros(len(index_time)),
                          'year': index_time.year,
                          'month': index_time.month,
                          'day':index_time.day}, 
                          index=index_time)
df_sp
for i in range(len(df_sp)):
    d = pos[(pos['year'] == df_sp['year'].iloc[i]) & (pos['month'] == df_sp['month'].iloc[i]) & (pos['day'] == df_sp['day'].iloc[i])]
    if not d.empty:
        df_sp['SentimentPolarity'].iloc[i] = np.round(float(d['SentimentPolarity']), 2)
df_sp

/home/mike/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,SentimentPolarity,year,month,day
2019-03-18,2.37,2019,3,18
2019-03-19,0.00,2019,3,19
2019-03-20,0.00,2019,3,20
2019-03-21,0.00,2019,3,21
2019-03-22,2.20,2019,3,22
...,...,...,...,...
2020-04-16,2.00,2020,4,16
2020-04-17,3.00,2020,4,17
2020-04-18,0.00,2020,4,18
2020-04-19,0.00,2020,4,19


In [26]:
fie = pd.to_datetime('2019-03-18')
ffe = pd.to_datetime('2020-04-20')
fip = ffe + pd.DateOffset(days=1)
ffp = ffe + pd.DateOffset(days=15)

In [30]:
df_sp

,SentimentPolarity,year,month,day
2019-03-18,2.37,2019,3,18
2019-03-19,0.00,2019,3,19
2019-03-20,0.00,2019,3,20
2019-03-21,0.00,2019,3,21
2019-03-22,2.20,2019,3,22
...,...,...,...,...
2020-04-16,2.00,2020,4,16
2020-04-17,3.00,2020,4,17
2020-04-18,0.00,2020,4,18
2020-04-19,0.00,2020,4,19


In [42]:
dfe = df_sp[fie:ffe]
dfe
dfdd = pd.DataFrame(dfe['SentimentPolarity'])  # dataframe to time series decomposition
decd = decompose(dfdd, period=28)  # time series decompositon


In [49]:
decd.trend.SentimentPolarity.iloc[0]

2.2958923823526516

In [55]:
for i in range(len(df_sp)):
    if df_sp.SentimentPolarity.iloc[i] == 0:
        df_sp['SentimentPolarity'].iloc[i] = decd.trend.SentimentPolarity.iloc[i]
        

In [60]:
dfe = df_sp[fie:ffe]
dfdd = pd.DataFrame(dfe['SentimentPolarity'])  # dataframe to time series decomposition
decd = decompose(dfdd, period=28)  # time series decompositon


In [61]:
pt = forecast(decd, steps=15, fc_func=drift)
pe = forecast(decd, steps=15, fc_func=drift, seasonal=True)

In [93]:
fig = go.Figure()
fig.layout.template = 'plotly_white'
fig.layout.yaxis.fixedrange = False
fig.layout.xaxis.fixedrange = True
fig.add_trace(go.Scatter(x=df_sp.index, y=df_sp.SentimentPolarity, mode='lines', name='Origin'))
fig.add_trace(go.Scatter(x=dfdd.index, y=decd.seasonal.SentimentPolarity, mode='lines', name='Season', line=dict(color="#ef5439")))
fig.add_trace(go.Scatter(x=dfdd.index, y=decd.resid.SentimentPolarity, mode='lines', name='Resid',  line=dict(color="#00cc96")))
fig.add_trace(go.Scatter(x=[ffe, fip], y=[decd.trend.SentimentPolarity[len(decd.trend.SentimentPolarity) - 1], pt.drift[0]], mode='lines',
                                 line=dict(color='red'), showlegend=False))
fig.add_trace(go.Scatter(x=dfdd.index, y=decd.trend.SentimentPolarity, mode='lines', name='Trend'))
fig.add_trace(go.Scatter(x=pt.index, y=pt.drift, mode='lines+markers', name='Trend-fcast', line=dict(color='red')))
fig.add_trace(go.Scatter(x=pe.index, y=pe['drift+seasonal'], mode='lines+markers', name='Predict',
                             line=dict( width=1, dash='dot')))
fig.update_layout(title='%s' % ("<b>" + 'Sentiment Polarity' + "</b>"), dragmode='pan',
                  margin={'l': 50, 'r': 100, 'b': 50, 't': 100}, width=900, height=500,
                  xaxis=dict(rangeslider=dict(visible=True)))
fig.update_xaxes(range=['2020-01-01', '2020-05-15']) #7 dias jornadas
fig.update_yaxes(range=[1.8, 3]) #7 dias jornadas
fig.write_image(""+"Sentiment Polarity predict"+".svg")
fig.show(config=dict({'scrollZoom': True}))
